In [1]:
''' 
    Impor Pustaka apa yang kita butuhkan
'''
import cv2
import mediapipe as mp
import numpy as np
import time

In [2]:
''' 
    Config Mediapipe Solution
'''
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(min_detection_confidence = 0.5, min_tracking_confidence=0.5)

mp_drawing = mp.solutions.drawing_utils

drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)

In [3]:
''' 
    Pojok Dir => Tempat untuk memeriksa metode/atribut
'''
print(dir(mp_face_mesh))
# print(dir(cv2))

['FACEMESH_CONTOURS', 'FACEMESH_FACE_OVAL', 'FACEMESH_IRISES', 'FACEMESH_LEFT_EYE', 'FACEMESH_LEFT_EYEBROW', 'FACEMESH_LEFT_IRIS', 'FACEMESH_LIPS', 'FACEMESH_NOSE', 'FACEMESH_NUM_LANDMARKS', 'FACEMESH_NUM_LANDMARKS_WITH_IRISES', 'FACEMESH_RIGHT_EYE', 'FACEMESH_RIGHT_EYEBROW', 'FACEMESH_RIGHT_IRIS', 'FACEMESH_TESSELATION', 'FaceMesh', 'NamedTuple', 'SolutionBase', '_BINARYPB_FILE_PATH', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'association_calculator_pb2', 'constant_side_packet_calculator_pb2', 'detections_to_rects_calculator_pb2', 'gate_calculator_pb2', 'image_to_tensor_calculator_pb2', 'inference_calculator_pb2', 'landmarks_refinement_calculator_pb2', 'logic_calculator_pb2', 'non_max_suppression_calculator_pb2', 'np', 'rect_transformation_calculator_pb2', 'split_vector_calculator_pb2', 'ssd_anchors_calculator_pb2', 'tensors_to_classification_calculator_pb2', 'tensors_to_detections_calculator_pb2', 'tensors_to_landmarks_c

In [4]:
nose_tip = [1]
right_eye = [33]
left_eye = [263]
lips_upper = [61, 291]
chin = [199]
targetted_landmark = []

targetted_landmark.extend(item for sublist in [nose_tip, right_eye, left_eye, lips_upper, chin] for item in sublist)

In [5]:
''' 
    Baca Kamera
'''
cap = cv2.VideoCapture(0)

# Loop utama untuk memproses setiap frame dari kamera
while cap.isOpened():
    # Baca frame terbaru dari kamera
    success, image = cap.read()
    start = time.time()
    
    # Balikkan gambar secara horizontal dan konversi ke RGB
    image = cv2.cvtColor(cv2.flip(image,1), cv2.COLOR_BGR2RGB)
    
     # Optimalkan kinerja dengan mengubah flag writeable gambar
    image.flags.writeable = False
    
    # Proses deteksi wajah dan landmark dengan MediaPipe
    results = face_mesh.process(image)
    
    # Proses deteksi wajah dan landmark dengan MediaPipe
    image.flags.writeable = True
    
    # Konversi gambar kembali ke format BGR
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    
    # Mendapatkan dimensi gambar
    img_h, img_w, img_c = image.shape
    face_3d = []
    face_2d = []
    
    # Cek apakah terdapat wajah yang terdeteksi
    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            for idx, lm in enumerate(face_landmarks.landmark):
                if idx in targetted_landmark:
                    if idx == 1:
                    # Dapatkan koordinat hidung dalam 2D dan 3D
                        nose_2d = (lm.x * img_w, lm.y * img_h)
                        nose_3d = (lm.x * img_w, lm.y * img_h, lm.z * 3000)
                    
                    x, y = int(lm.x * img_w), int(lm.y * img_h)
                    
                    # Dapatkan koordinat 2D
                    face_2d.append([x, y])
                    # Dapatkan koordinat 3D
                    face_3d.append([x, y, lm.z])
                    
            # Konversi koordinat menjadi array NumPy
            face_2d = np.array(face_2d, dtype=np.float64)
            face_3d = np.array(face_3d, dtype=np.float64)
            
            # Matriks kamera
            focal_length = 1 * img_w
            cam_matrix = np.array([
                                    [focal_length, 0, img_h / 2],
                                    [0, focal_length, img_w / 2],
                                    [0, 0, 1]
                                ])
            dist_matrix = np.zeros((4, 1), dtype=np.float64)
            
            # Hitung orientasi kepala menggunakan solvePnP
            success, rot_vec, trans_vec = cv2.solvePnP(face_3d, face_2d, cam_matrix, dist_matrix)
            # Matriks Rotasi
            rmat, jac = cv2.Rodrigues(rot_vec)
            # Angle
            angles, mtxR, mtxQ, Qx, Qy, Qz = cv2.RQDecomp3x3(rmat)
            
            # Mendapatkan Derajat Rotasi Y
            x = angles[0] * 360
            y = angles[1] * 360
            z = angles[2] * 360
            
            # Melihat kemiringan kepala pengguna
            if y < -10:
                text = 'Looking Left'
            elif y > 10:
                text = 'Looking Right'
            elif x < -7:
                text = 'Looking Down'
            elif x > 10:
                text = 'Looking Up'
            else:
                text = 'Forward'
                
            # Memproyeksikan koordinat hidung ke 3d
            nose_3d_projection, jacobian = cv2.projectPoints(nose_3d, rot_vec, trans_vec, cam_matrix, dist_matrix)
            # Gambar Arah Pandangan
            p1 = (int(nose_2d[0]), int(nose_2d[1]))
            p2 = (int(nose_2d[0] + y * 10), int(nose_2d[1] - x * 10))
            cv2.line(image, p1, p2, (255, 0, 0), 3)
            
            
            # Tambahkan text pada gambar
            cv2.putText(image, text, (20,50), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 2)
            cv2.putText(image, "x: " + str(np.round(x,2)), (500, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(image, "y: " + str(np.round(y,2)), (500, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(image, "z: " + str(np.round(z,2)), (500, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            
            # Hitung FPS
            end = time.time()
            totalTime = end - start
            fps = 1 / totalTime
            cv2.putText(image, f"FPS: {int(fps)}", (20, 450), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            
            # Gambar Landmark Wajah pada gambar
            mp_drawing.draw_landmarks(
                image = image,
                landmark_list = face_landmarks,
                connections = mp_face_mesh.FACEMESH_CONTOURS,
                landmark_drawing_spec = drawing_spec,
                connection_drawing_spec = drawing_spec
            )
            
            cv2.imshow('Head Pose Estimation', image)
            
            if cv2.waitKey(1) == ord('q'):
                break
            
cap.release()

KeyboardInterrupt: 

In [1]:

cap.release()   
cv2.destroyAllWindows()

NameError: name 'cap' is not defined